In [1]:
import json
import tensorflow as tf
import numpy as np
from sklearn.cross_validation import train_test_split
from tensor2tensor.utils import beam_search

/home/jupyter/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
with open('dictionary.json') as fopen:
    x = json.load(fopen)
dictionary_english = x['english']
dictionary_bahasa = x['bahasa']

In [3]:
with open('english-malay.json') as fopen:
    x = json.load(fopen)
english = x[0]
bahasa = x[1]

In [4]:
GO = dictionary_english['dictionary']['GO']
PAD = dictionary_english['dictionary']['PAD']
EOS = dictionary_english['dictionary']['EOS']
UNK = dictionary_english['dictionary']['UNK']

In [5]:
from tqdm import tqdm

for i in tqdm(range(len(bahasa))):
    bahasa[i].append('EOS')

100%|██████████| 100000/100000 [00:00<00:00, 1075509.45it/s]


In [10]:
def embed_seq(x, vocab_sz, embed_dim, name, zero_pad=True): 
    embedding = tf.get_variable(name, [vocab_sz, embed_dim]) 
    if zero_pad:
        embedding = tf.concat([tf.zeros([1, embed_dim]), embedding[1:, :]], 0) 
    x = tf.nn.embedding_lookup(embedding, x)
    return x

def position_encoding(inputs):
    T = tf.shape(inputs)[1]
    repr_dim = inputs.get_shape()[-1].value
    pos = tf.reshape(tf.range(0.0, tf.to_float(T), dtype=tf.float32), [-1, 1])
    i = np.arange(0, repr_dim, 2, np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1])

def layer_norm(inputs, epsilon=1e-8):
    mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
    normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))
    params_shape = inputs.get_shape()[-1:]
    gamma = tf.get_variable('gamma', params_shape, tf.float32, tf.ones_initializer())
    beta = tf.get_variable('beta', params_shape, tf.float32, tf.zeros_initializer())
    return gamma * normalized + beta

def cnn_block(x, dilation_rate, pad_sz, hidden_dim, kernel_size):
    x = layer_norm(x)
    pad = tf.zeros([tf.shape(x)[0], pad_sz, hidden_dim])
    x =  tf.layers.conv1d(inputs = tf.concat([pad, x, pad], 1),
                          filters = hidden_dim,
                          kernel_size = kernel_size,
                          dilation_rate = dilation_rate)
    x = x[:, :-pad_sz, :]
    x = tf.nn.relu(x)
    return x

def Attention(Q, inputs, num_units, num_heads = 8, activation = None):
    inputs = tf.layers.dropout(inputs, 0.1, training=True)
    T_q = tf.shape(Q)[1]
    T_k = tf.shape(inputs)[1]
    K_V = tf.layers.dense(inputs, 2*num_units, activation)
    K, V = tf.split(K_V, 2, -1)
    Q_ = tf.concat(tf.split(Q, num_heads, axis=2), 0)
    K_ = tf.concat(tf.split(K, num_heads, axis=2), 0)
    V_ = tf.concat(tf.split(V, num_heads, axis=2), 0)
    p_gen = tf.layers.dense(K * V, 1)
    p_gen = tf.sigmoid(p_gen)
    align = tf.matmul(Q_, K_, transpose_b=True)
    align *= tf.rsqrt(tf.to_float(K_.get_shape()[-1].value))
    paddings = tf.fill(tf.shape(align), float('-inf'))
    lower_tri = tf.ones([T_q, T_k])
    lower_tri = tf.linalg.LinearOperatorLowerTriangular(lower_tri).to_dense()
    masks = tf.tile(tf.expand_dims(lower_tri,0), [tf.shape(align)[0],1,1])
    align = tf.where(tf.equal(masks, 0), paddings, align)
    align = tf.nn.softmax(align)
    alignments = tf.transpose(align, [0, 2, 1]) 
    x = tf.matmul(align, V_)
    x = tf.concat(tf.split(x, num_heads, axis=0), 2)
    x += Q
    x = layer_norm(x)
    return x, alignments, p_gen

class Model:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 from_dict_size, to_dict_size, learning_rate, 
                 kernel_size = 2, n_attn_heads = 16):

        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])

        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype = tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype = tf.int32)
        batch_size = tf.shape(self.X)[0]
        self.batch_size = batch_size
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        self.encoder_embedding = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        self.decoder_embedding = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        
        self.num_layers = num_layers
        self.kernel_size = kernel_size
        self.size_layer = size_layer
        self.n_attn_heads = n_attn_heads
        self.dict_size = to_dict_size
        
        self.training_logits = self.forward(self.X, decoder_input)

        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
    def forward(self, x, y, reuse = False):
        with tf.variable_scope('forward',reuse=reuse):
            with tf.variable_scope('forward',reuse=reuse):
                encoder_embedded = tf.nn.embedding_lookup(self.encoder_embedding, x)
                decoder_embedded = tf.nn.embedding_lookup(self.decoder_embedding, y)
                encoder_embedded += position_encoding(encoder_embedded)

                for i in range(self.num_layers): 
                    dilation_rate = 2 ** i
                    pad_sz = (self.kernel_size - 1) * dilation_rate 
                    with tf.variable_scope('block_%d'%i,reuse=reuse):
                        encoder_embedded += cnn_block(encoder_embedded, dilation_rate, 
                                                      pad_sz, self.size_layer, self.kernel_size)

                g = tf.identity(decoder_embedded)
                for i in range(self.num_layers):
                    dilation_rate = 2 ** i
                    pad_sz = (self.kernel_size - 1) * dilation_rate
                    with tf.variable_scope('decode_%d'%i,reuse=reuse):
                        attn_res = h = cnn_block(decoder_embedded, dilation_rate, 
                                                 pad_sz, self.size_layer, self.kernel_size)
                        with tf.variable_scope('attention_%d'%i,reuse=reuse):
                            h, alignment, p_gen = Attention(attn_res, encoder_embedded, self.size_layer)
                        
                        vocab_dist = tf.nn.softmax(encoder_embedded) * p_gen
                        alignments = alignment[0] * (1 - p_gen)
                        encoder_embedded += vocab_dist
                        decoder_embedded += h

                return tf.layers.dense(decoder_embedded, self.dict_size)

In [11]:
size_layer = 256
num_layers = 4
embedded_size = 256
learning_rate = 1e-3
batch_size = 128
epoch = 20

In [12]:
def beam_search_decoding(length = 20, beam_width = 5):
    initial_ids = tf.fill([model.batch_size], GO)
    
    def symbols_to_logits(ids):
        x = tf.contrib.seq2seq.tile_batch(model.X, beam_width)
        logits = model.forward(x, ids, reuse = True)
        return logits[:, tf.shape(ids)[1]-1, :]

    final_ids, final_probs, _ = beam_search.beam_search(
        symbols_to_logits,
        initial_ids,
        beam_width,
        length,
        len(dictionary_bahasa['dictionary']),
        0.0,
        eos_id = EOS)
    
    return final_ids

In [13]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer, num_layers, embedded_size, 
                len(dictionary_english['dictionary']), len(dictionary_bahasa['dictionary']),
                                                           learning_rate)
model.generate = beam_search_decoding()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use keras.layers.conv1d instead.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [14]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i:
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

In [15]:
english = str_idx(english, dictionary_english['dictionary'])
bahasa = str_idx(bahasa, dictionary_bahasa['dictionary'])

In [16]:
train_X, test_X, train_Y, test_Y = train_test_split(english, bahasa, test_size = 0.2)

In [17]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [18]:
import time

for EPOCH in range(20):
    lasttime = time.time()

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x, _ = pad_sentence_batch(train_X[i : index], PAD)
        batch_y, _ = pad_sentence_batch(train_Y[i : index], PAD)
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x, _ = pad_sentence_batch(test_X[i : index], PAD)
        batch_y, _ = pad_sentence_batch(test_Y[i : index], PAD)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )

train minibatch loop:   0%|          | 2/625 [00:00<00:58, 10.73it/s, accuracy=0.344, cost=4.03]

time taken: 87.67448663711548
epoch: 0, training loss: 5.059138, training acc: 0.247874, valid loss: 4.379347, valid acc: 0.322134



train minibatch loop:   0%|          | 2/625 [00:00<00:57, 10.87it/s, accuracy=0.408, cost=3.46]

time taken: 79.96844244003296
epoch: 1, training loss: 3.928127, training acc: 0.350812, valid loss: 4.117769, valid acc: 0.356321



train minibatch loop:   0%|          | 2/625 [00:00<00:57, 10.84it/s, accuracy=0.444, cost=3.1]

...

time taken: 80.38998436927795
epoch: 17, training loss: 1.893885, training acc: 0.594640, valid loss: 4.814614, valid acc: 0.371967



train minibatch loop:   0%|          | 2/625 [00:00<00:57, 10.76it/s, accuracy=0.631, cost=1.64]

time taken: 80.34107398986816
epoch: 18, training loss: 1.852196, training acc: 0.600806, valid loss: 4.868435, valid acc: 0.368715



test minibatch loop: 100%|██████████| 157/157 [00:08<00:00, 18.43it/s, accuracy=0.404, cost=4.31]

time taken: 80.15598678588867
epoch: 19, training loss: 1.810724, training acc: 0.607612, valid loss: 4.894094, valid acc: 0.365983



In [24]:
dictionary_bahasa['rev_dictionary'] = {int(k): v for k, v in dictionary_bahasa['rev_dictionary'].items()}
dictionary_english['rev_dictionary'] = {int(k): v for k, v in dictionary_english['rev_dictionary'].items()}

### Let we test our beam search

This is our english string,

In [26]:
' '.join([dictionary_english['rev_dictionary'][i] for i in test_X[0]])

'from the taliban point of view that was their number one mistake last time'

Predicted bahasa translation from english string,

In [17]:
t = sess.run(model.generate, feed_dict = {model.X: [test_X[0]]})[0,0,:]

In [23]:
' '.join([dictionary_bahasa['rev_dictionary'][i] for i in t])

'GO dari taliban menetapkan pandangan yang saya adalah nombor yang ditimbulkan sepanjang masa satu kali terakhir EOS PAD PAD'

Actual translation,

In [27]:
' '.join([dictionary_bahasa['rev_dictionary'][i] for i in test_Y[0]])

'dari sudut pandang taliban apos itulah kesilapan nombor satu mereka kali terakhir EOS'